In [1]:
import netket as nk
import numpy as np
from scripts import functions as f

/home/keisuke/Documents/Research/sshfs/DimerMaster/scripts/functions.py:65: NumbaDeprecationWarning: The 'numba.jitclass' decorator has moved to 'numba.experimental.jitclass' to better reflect the experimental nature of the functionality. Please update your imports to accommodate this change and see https://numba.pydata.org/numba-doc/latest/reference/deprecation.html#change-of-jitclass-location for the time frame.
  @jitclass(spec)


In [44]:
from importlib import reload

In [9]:
from netket.machine import functions2 as f2

In [69]:
reload(f2)

<module 'netket.machine.functions2' from '/home/keisuke/Documents/Research/sshfs/DimerMaster/netket/machine/functions2.py'>

In [70]:
h = f2.new_hex(np.array([4,4]))

In [56]:
coordinate = h.lattice_coor

# return_array = self.coor_to_lattice_num(self.translation(coordinate, self.all_unit_cell))
return_array_coor = h.translation(coordinate, h.all_unit_cell)

In [62]:
return_array_coor_ = return_array_coor.copy()
return_array_coor_ = h.riverse(return_array_coor_)

In [68]:
np.concatenate((return_array_coor, return_array_coor),axis=0 )

(16, 32, 2)

In [18]:
h.coor_to_lattice_num(h.lattice_coor)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31])

In [4]:
from importlib import reload

In [59]:
reload(f)

<module 'functions' from '/home/keisuke/Documents/Research/sshfs/DimerMaster/functions.py'>

In [3]:
h = new_hex(np.array([4,4]))

In [7]:
h.all_unit_cell

array([[0, 0],
       [1, 0],
       [0, 1],
       [1, 1],
       [0, 2],
       [1, 2],
       [0, 3],
       [1, 3]])

In [61]:
e, ec, pe, pec = h.for_hamiltonian()

In [62]:
coordinate = h.lattice_coor
h.all_hex_index
h.coor_to_lattice_num(h.translation(coordinate, h.all_unit_cell))

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15],
       [ 2,  3,  0,  1,  6,  7,  4,  5, 10, 11,  8,  9, 14, 15, 12, 13],
       [ 8,  9, 10, 11, 12, 13, 14, 15,  0,  1,  2,  3,  4,  5,  6,  7],
       [10, 11,  8,  9, 14, 15, 12, 13,  2,  3,  0,  1,  6,  7,  4,  5]])

In [63]:
h.autom

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15],
       [ 2,  3,  0,  1,  6,  7,  4,  5, 10, 11,  8,  9, 14, 15, 12, 13],
       [ 8,  9, 10, 11, 12, 13, 14, 15,  0,  1,  2,  3,  4,  5,  6,  7],
       [10, 11,  8,  9, 14, 15, 12, 13,  2,  3,  0,  1,  6,  7,  4,  5]])

In [64]:
pe[0][0,0]

array([11,  7])

In [74]:
op = f.dimer_flip()

In [75]:
op.get_conn(np.ones(16))

(array([[ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1],
        [-1,  1,  1,  1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1],
        [ 1,  1,  1,  1,  1,  1,  1,  1, -1,  1,  1,  1, -1,  1,  1,  1],
        [ 1,  1, -1,  1,  1,  1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1],
        [ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1,  1,  1,  1, -1,  1]],
       dtype=int8),
 array([ 0.+0.j, -1.+0.j, -1.+0.j, -1.+0.j, -1.+0.j]))

In [8]:
from numba import njit

In [12]:
@njit
def convert(x):
    
    y = np.ones(10, dtype=np.int8)
    return y.astype(np.float64)

In [20]:
length = [4,2]
V = 1
h = 1
sigmaz = np.array([[1, 0], [0, -1]])
sigmax = np.array([[0,1],[1,0]])
sigmay = np.array([[0,1],[-1,0]])

mszsz = np.kron(sigmaz, sigmaz)
mszsx = np.kron(sigmax, sigmax)
mszsy = np.kron(sigmay, sigmay)

potential_anti = (np.identity(4) + mszsz)/2
potential_ferro = (np.identity(4) - mszsz)/2

# hexagon = nk.machine.graph_hex(length = length)
hexagon = f.new_hex(np.array(length))


g = nk.graph.Graph(nodes = [i for i in range(length[0] * length[1] * 2)])

e, ec, ce, cec = hexagon.for_hamiltonian()

hi = nk.hilbert.Spin(s=0.5, graph=g)
op = nk.operator.DimerLocalOperator(hi)

for edge, edge_color, pe, pec in zip(e, ec, ce, cec):

    l_op = -h*mszsx

    l_op = np.kron(np.identity(2), l_op)
    l_op = np.kron(l_op, np.identity(2))

    mat = []
    edge_ = []
    for p, c in zip(pe, pec):
        mat.append(np.kron(
            potential_ferro if c[0] == 1 else potential_anti,
            potential_ferro if c[1] == 1 else potential_anti,
        ))
        edge_.append(p[0].tolist() + p[1].tolist())

    op += nk.operator.LocalOperator(hi, l_op @ mat[0] + V * mat[0], edge_[0])
    op += nk.operator.LocalOperator(hi, l_op @ mat[1] + V * mat[1], edge_[1])


In [31]:
np.random.randint(0,3,10)

array([1, 1, 0, 0, 0, 1, 1, 0, 0, 2])